### The Culture of International Relations

#### About this project
Cultural treaties are the bi-lateral and multilateral agreements among states that promote and regulate cooperation and exchange in the fields of life generally call cultural or intellectual. Although it was only invented in the early twentieth century, this treaty type came to be the fourth most common bilateral treaty in the period 1900-1980 (Poast et al., 2010). In this project, we seek to use several (mostly European) states’ cultural treaties as a historical source with which to explore the emergence of a global concept of culture in the twentieth century. Specifically, the project will investigate the hypothesis that the culture concept, in contrast to earlier ideas of civilization, played a key role in the consolidation of the post-World War II international order.

The central questions that interest me here can be divided into two groups: 
- First, what is the story of the cultural treaty, as a specific tool of international relations, in the twentieth century? What was the historical curve of cultural treaty-making? For example, in which political or ideological constellations do we find (the most) use of cultural treaties? Among which countries, in which historical periods? What networks of relations were thereby created, reinforced, or challenged? 
- Second, what is the "culture" addressed in these treaties? That is, what do the two signatories seem to mean by "culture" in these documents, and what does that tell us about the role that concept played in the international system? How can quantitative work on this dataset advance research questions about the history of concepts?

In this notebook, we deal with these treaties in three ways:
1) quantitative analysis of "metadata" about all bilateral cultural treaties signed betweeen 1919 and 1972, as found in the World Treaty Index or WTI (Poast et al., 2010).
    For more on how exactly we define a "cultural treaty" here, and on other principles of selection, see... [add this, using text now in "WTI quality assurance"].
2) network analysis of the system of international relationships created by these treaties (using data from WTI, as above).
3) Text analysis of the complete texts of selected treaties. 

After some set-up sections, the discussion of the material begins at "Part 1," below.

### Brief Instructions on Jupyter Notebooks
Please see [this tutorial](https://www.youtube.com/watch?v=h9S4kN4l5Is) for an introduction on what Jupyter notebooks are and how to use them. There are lots of other Jupyter tutorials on YouTube (and elsewhere) as well. In short, a notebook is a document with embedded executable code presented in a simple and easy to use web interface. Most important things to note are:
- Click on the menu Help -> User Interface Tour for an overview of the Jupyter Notebook App user interface.
- The **code cells** contains the script code (Python in this case, but can be other languages are also suported) and are the sections marked by **In [x]** in the left margin. It is marked as **In []** if it hasn't been executed, and as **In [n]** when it has been executed(n is an integer). A cell marked as **In [\*]** is either executing, or waiting to be executed (i.e. other cells are executing).
- The **current cell** is highlighted with a blue (or green if in "edit" mode) border. You make a cell current by clicking on it,
- Code cells aren't executed automatically. Instead you execute the current cell by either pressing **shift+enter** or the **play** button in the toolbar. The output (or result) of a cell's execution is presented directly below the cell prefixed by **Out[n]**.
- The next cell will automatically be selected (made current) after a cell has been executed. Repeatadly pressing **shift+enter** or the play button hence executes the cells in sequence.
- You can run the entire notebook in a single step by clicking on the menu Cell -> Run All. Note that this can take some time to finish. You can see how cells are executed in sequence via the indicator in the margin (i.e. "In [\*]" changes to "In [n]" where n is an integer).
- The cells can be edited if they are double-clicked, in which case the cell border turns green. Use the ESC key to escape edit mode (or click on any other cell).

To restart the kernel (i.e. the computational engine assigned to your session), click on the menu Kernel -> Restart. 


In [ ]:
%%html
<style>
.jupyter-widgets { font-size: 8pt; }
.widget-label { font-size: 8pt;}
.widget-vbox .widget-label { width: 60px; }
.widget-dropdown > select { font-size: 8pt; }
.widget-select > select { font-size: 8pt; }
.widget-toggle-buttons .widget-toggle-button {
    font-size: 8pt;
    width: 100px;
}
</style>

### <span style='color:blue'>**Mandatory Prepare Step**</span>: Setup Notebook and Load and Process Treaty Master Index
The following code cell to be executed once for each user session. The step loads utility Python code stored in separate files, and imports dependencies to external libraries. The code also loads the WTI master index (and some related data files), and prepares the data for subsequent use.

The treaty data is processed as follows:
- All the treaty data are loaded.Extract year treaty was signed as seperate fields
- Add new fields for specified signed period divisions
- Fields 'group1' and 'group2' are ignored (many missing values). Instead group are fetched via party code from encoding found in the "groups" table.

In [ ]:
# Setup
%load_ext autoreload
%autoreload 2

import os
import warnings
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib.ticker import MaxNLocator

from IPython.display import display, HTML

os.sys.path = os.sys.path if '..' in os.sys.path else os.sys.path + ['..']
warnings.filterwarnings('ignore')

import configuration_elements as config

from common.file_utility import FileUtility
from common.widgets_utility import BaseWidgetUtility
from common.utility import extend, flatten, getLogger, StaticColorMap
from common.treaty_state import load_treaty_state

logger = getLogger(name='cultural_treaties')
state = load_treaty_state('../data')
golden_ratio = 1.618


### Chart: Treaty Quantities by Selected Parties 
This chart displays the number of treaties per time period for each party or group of parties. The time period can be year or one of a predefined set of time period divisions. The default time period division is 1919-1944, 1945-1955, 1956-1966, and 1967-1972, the alternative division has 1940-1944 as an additional period. The third division is a single period between 1945 and 1972 (inclusive)

Several parties can be selected using CTRL + click on left mouse button. Shift-Up/Down can also be used to select consecutive parties. Some predefined party sets exist in the "Preset" dropdown, and, when selected, the corresponding parties is selected in the "Parties" widget. Use the "Top #n" slider to display parties with most treaties for each period. Note that a value greater than 0 will disable the "Parties" widget since these to selections are mutually exclusive. Set slider back to 0 to enable the "Parties" widget.

The treaty count is based on the following rules:
- Treaties outside of selected division are discarded
- When "Is Cultural" is selected than the only treaties included are those having "is cultural" flag in WTI index set to "Yes" 
- When "Topic is 7CULT" is selected than all treaties having topic equal to 7CULT are included using original topic value according to WTI i.e. no recoding!
- Candidate for removal: When "No filter" is selected then no topic filter is applied

The topic filter also restricts the extra category count added when "Other category" or "All category" are selected. When "Other" is selected, than the "complement" count of the sum of all countries not included is added as a category, and the "All" category adds the count of all treaties. Note again, that both these counts are restricted by time period (no treaty outside period), and selected topic filter.

Also note that when several countries are selected, than the count for a given country is the count of all treaties that country signed for each time period (filtered by topic selection). Each of the treaties that are signed *between* two of the selected parties will hence add +1 for each of the two parties.



In [ ]:
# Code
%matplotlib inline
import bokeh
colors = bokeh.palettes.Category20[20]

def plot_treaties_per_period(data, output_format, plot_style, figsize=(12,6), xlabel='', ylabel='', xticks=None, normalized=False):
    matplotlib.style.use(plot_style)
    stacked = 'stacked' in output_format
    kind = output_format.split('_')[1]
    ax = data.plot(kind=kind, stacked=stacked, figsize=figsize, color=colors)
    
    if xticks is not None:
        ax.set_xticks(xticks)
    
    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    if normalized:
        ax.set_ylim([0, 100])
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
    
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    
    # Put a legend to the right of the current axis
    legend = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    legend.get_frame().set_linewidth(0.0)

    for tick in ax.get_xticklabels():        
        tick.set_rotation(45)
    
def get_top_parties(stacked_treaties, period_group, party_name, n_top=3):
    # data = stacked_treaties.merge(state.parties, how='inner', left_on='party', right_index=True)
    xd = stacked_treaties.groupby([period_group, party_name]).size().rename('TopCount').reset_index()
    top_list = xd.groupby([period_group]).apply(lambda x: x.nlargest(n_top, 'TopCount'))\
        .reset_index(level=0, drop=True)\
        .set_index([period_group, party_name])
    return top_list

def get_treaty_subset(treaties, period_group, treaty_filter):
    
    treaties_within_division = treaties[treaties[period_group]!='other']

    treaty_subset = treaties_within_division.loc[(treaties_within_division.is_cultural==True)] if treaty_filter == 'is_cultural'\
        else (treaties_within_division.loc[(treaties_within_division.topic1=='7CULT')]  if treaty_filter == 'is_7cult' else treaties_within_division)
    
    return treaty_subset

def get_treaties_of_interest(state=None, period_group='signed_period', party_name='party_name', parties=None, treaty_filter='', extra_category='', n_top=0):

    treaty_subset = get_treaty_subset(state.treaties, period_group, treaty_filter)

    # Skapa urvalet från stacked_treaties så att vi kan gruppera på valda parties via column "party"
    # Regel: Filterera ut på treaty_ids, och forcera att "party" måste finnas i valda parter (vi vill inte gruppera på motpart såvida den inte finns i parties)
    # Ger överträffar för valda parties som har fördrag mellan varandra

    df = state.stacked_treaties.loc[treaty_subset.index] #  state.stacked_treaties[state.stacked_treaties.index.isin(treaty_subset.index)]

    if isinstance(parties, list) and len(parties) > 0:
        treaty_ids = treaty_subset[(treaty_subset.party1.isin(parties))|((treaty_subset.party2.isin(parties)))].index
        treaties_of_parties = df[df.index.isin(treaty_ids)&df.party.isin(parties)] # de avtal som vars länder valts
    else:
        df_top = get_top_parties(df, period_group=period_group, party_name=party_name, n_top=n_top).drop(['TopCount'], axis=1)
        treaties_of_parties = df.merge(df_top, how='inner', left_on=[period_group, party_name], right_index=True)

    df_extra = None
    if extra_category == 'other_category':
        extra_ids = treaty_subset[~treaty_subset.index.isin(treaties_of_parties.index)].index
        df_extra = df[df.index.isin(extra_ids)&(df.reversed==False)]
        extra_party = state.get_party('ALL OTHER')
        
    elif extra_category == 'all_category':
        df_extra = df[df.index.isin(treaty_subset.index)&(df.reversed==False)]                        
        extra_party = state.get_party('ALL')

    if df_extra is not None:
        df_extra = df_extra.assign(party=extra_party['party'],
                                   party_name=extra_party['party_name'],
                                   party_short_name=extra_party['short_name'],
                                   party_country=extra_party['country'])
        treaties_of_parties = pd.concat([treaties_of_parties, df_extra])
    else:
        print("df_extra is None")
    return treaties_of_parties.groupby([period_group, party_name]).size().reset_index()\
            .rename(columns={ period_group: 'Period', party_name: 'Party', 0: 'Count' })
    
def display_treaties_per_period(
    period_group,
    party_name,
    parties,
    treaty_filter='',
    extra_category='',
    normalize_values=False,
    output_format='chart',
    plot_style='classic',
    top_n_parties=5
    ):
        
    try:
        parties = list(parties)
        
        data = get_treaties_of_interest(state, period_group=period_group, party_name=party_name, parties=parties,
                                        treaty_filter=treaty_filter, extra_category=extra_category, n_top=top_n_parties)
        
        if data.shape[0] == 0:
            print('No data for selection')
            return

        pivot = pd.pivot_table(data, index=['Period'], values=["Count"], columns=['Party'], fill_value=0)
        pivot.columns = [ x[-1] for x in pivot.columns ]
    
        if period_group == 'signed_year':
            missing_years = [ x for x in range(data.Period.min(), data.Period.max() + 1) if x not in pivot.index ]
            for year in missing_years:
                pivot.loc[year] = len(pivot.columns) * [0]
            pivot.sort_index(axis=0, inplace=True)
    
        if normalize_values is True:
            pivot = pivot.div(0.01 * pivot.sum(1), axis=0)

        if output_format.startswith('plot'):

            label = 'Number of treaties' if not normalize_values else 'Share%'

            ylabel = label if 'barh' not in output_format else ''
            xlabel = label if 'barh' in output_format else ''

            height = 10 if 'barh' in output_format and period_group == 'signed_year' else 6

            xticks = list(range(data.Period.min(), data.Period.max() + 1)) if 'line' in output_format and period_group == 'signed_year' else None
            
            plot_treaties_per_period(pivot, output_format, plot_style,
                                     figsize=(18, 18.0/(2*golden_ratio)), xlabel=xlabel, ylabel=ylabel, xticks=xticks,
                                     normalized=normalize_values)

        elif output_format == 'table':
            display(data)
            #display(HTML(data.to_html()))
        else:
            display(pivot)
            
    except Exception as ex:
        logger.error(ex)
        raise

def treaty_quantities_by_selected_parties_main():
    
    tw = BaseWidgetUtility(
        period_group=widgets.Dropdown(
            options= { x['title']: x['column'] for x in state.period_specification },
            value='signed_period',
            description='Divisions',
            layout=widgets.Layout(width='200px')
        ),
        party_name=widgets.Dropdown(
            options={
                'WTI Code': 'party',
                'WTI Name': 'party_name',
                'WTI Short': 'party_short_name',
                'Country': 'party_country'
            },
            value='party_name',
            description='Name',
            layout=widgets.Layout(width='200px')
        ),
        normalize_values=widgets.ToggleButton(
            description='Display %',
            icon='',
            layout=widgets.Layout(width='100px', left='0')
        ),
        output_format=widgets.Dropdown(
            description='Output',
            options=config.output_formats,
            value='plot_bar_stacked',
            layout=widgets.Layout(width='200px')
        ),
        plot_style=widgets.Dropdown(
            options=config.matplotlib_plot_styles, value='seaborn-pastel',
            description='Style',
            layout=widgets.Layout(width='200px')
        ),
        top_n_parties=widgets.IntSlider(
            value=0, min=0, max=10, step=1,
            description='Top #',
            continuous_update=False,
            layout=widgets.Layout(width='200px')
        ),
        party_preset=widgets.Dropdown(
            options={
                'PartyOf5': config.parties_of_interest,
                'Germany (all)': [ 'GERMU', 'GERMAN', 'GERME', 'GERMW', 'GERMA' ]
            },
            value=None,
            description='Presets',
            layout=widgets.Layout(width='200px')
        ),
        parties=widgets.SelectMultiple(
            options=state.get_countries_list(),
            value=['FRANCE'],
            rows=12,
            description='Parties',
            disabled=False,
            layout=widgets.Layout(width='180px')
        ),
        treaty_filter=widgets.ToggleButtons(
            options={ 'Is Cultural': 'is_cultural', 'Topic is 7CULT': 'is_7cult', 'No filter': '' },
            description='Topic filter:',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltips=[
                'Include ONLY treaties marked as "is cultural"',
                'Include all treaties where topic is 7CULT (disregard "is cultural" flag)',
                'Include ALL treaties (no topic filter)'
            ],
            value='is_cultural',
            layout=widgets.Layout(width='200px')
        ),
        extra_category=widgets.ToggleButtons(
            options={ 'Other category': 'other_category', 'All category': 'all_category', 'Nothing': '' },
            description='Include:',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            value='',
            layout=widgets.Layout(width='200px')
        )
    )

    itw = widgets.interactive(
        display_treaties_per_period,
        period_group=tw.period_group,
        party_name=tw.party_name,
        parties=tw.parties,
        treaty_filter=tw.treaty_filter,
        extra_category=tw.extra_category,
        normalize_values=tw.normalize_values,
        output_format=tw.output_format,
        plot_style=tw.plot_style,
        top_n_parties=tw.top_n_parties
    )

    def on_party_preset_change(change):
        if len(tw.party_preset.value or []) == 0:
            return
        if tw.top_n_parties.value > 0:
            tw.top_n_parties.value = 0
        tw.parties.value = tw.party_preset.value

    tw.party_preset.observe(on_party_preset_change, names='value')

    def on_parties_change(change):
        try:
            tw.top_n_parties.unobserve(on_top_n_parties_change, names='value')
            if tw.top_n_parties.value != 0:
                tw.top_n_parties.value = 0
            tw.top_n_parties.observe(on_top_n_parties_change, names='value')
        except Exception as ex:
            logger.info(ex)
            
    def on_top_n_parties_change(change):
        try:
            if tw.top_n_parties.value > 0:
                tw.parties.unobserve(on_parties_change, names='value')
                tw.parties.disabled = True
                if len(tw.parties.value) > 0:
                    tw.parties.value = []
            else:
                tw.parties.observe(on_parties_change, names='value')
                tw.parties.disabled = False
        except Exception as ex:
            logger.info(ex)

    tw.parties.observe(on_parties_change, names='value')
    tw.top_n_parties.observe(on_top_n_parties_change, names='value')

    column_boxes =  widgets.HBox([
        widgets.VBox([tw.period_group, tw.party_name, tw.top_n_parties, tw.party_preset]),
        widgets.VBox([ tw.parties ]),
        widgets.VBox([ tw.treaty_filter, tw.extra_category]),
        widgets.VBox([ tw.output_format, tw.plot_style]),
        widgets.VBox([ tw.normalize_values ])
    ])
    display(widgets.VBox([column_boxes, itw.children[-1]]))
    itw.update()

treaty_quantities_by_selected_parties_main()


###  Chart: Treaty Quantities by Selected Topics
This chart displays the number of treaties per topic category for each time period. Only treaties signed by one of the selected parties are included in the count (unless either of ALL OTHER or ALL is selected, see below).

See previous chart for a description of "period" and "parties" options (the same rules apply in this chart).

Topics **not** specified in the selected "Category" are as default **excluded** from the counts. If the "Add OTHER topics" toggle is set then a new "OTHER" category is added as a summed up count for topics outside the selected topic category. 

Note that, as default when the "Recode 7CORR" is set, all treaties marked as "is_cultural" = "Yes" in the WTI index are assigned a new topic code "7CORR". The remaining treaties ´having topic 7CULT are not recoded (will keep this code).

When the "Chart per treaty" toggle is set, then a chart is created for each party. Please not that this might take some time if many countries are selected. Extra charts will also be created for **excluded** parties (i.e. parties not selected) if either of the "ALL OTHER" or "ALL" party items are selected. The "ALL OTHER" will include not selected partiies, and "ALL" will include all parties. In both cases the category filter is applies as for any other chart.

The categories are defined in the code cell below, and can easily be changed in "category_group_settings". The structure of the specification is
```python
category_group_settings = {
    'category-group-name': {
        'category-item-1': [list-of-topic-codes-in-category-1],
        'category-item-2': [list-of-topic-codes-in-category-2],
        ...
        'category-item-n': [list-of-topic-codes-in-category-n]
    },
```
The **category-group-name**s will be visible in the "Category" dropdown list, and it is hence important for these names to be unique. It can be an arbitrary name. The **category-item**s is the name that will be visible on the chart legend as label for associated list of topic-codes. The **list-of-topic-codes**, finally, must be a list of valid topic codes in the WTI-index (+ 7CORR). Note that these lists within the same category-group should be mutually exclusive i.e. the same code should not be added to more than one group (will give an unpredicted result). Also note that correct braces **must** be used, and strings **must** be surrounded by "'", and all topic codes must also be in uppercase. Follow the same syntax as in the existing specifications. 



In [ ]:
# Code
%matplotlib inline
import matplotlib
from IPython.display import clear_output
import ipywidgets as widgets

category_group_settings = {
    '7CORR, 7SCIEN, and 7EDUC': {
        '7CORR': ['7CORR'],
        '7SCIEN': ['7SCIEN'],
        '7EDUC': ['7EDUC']
    },
    '7CORR, 7SCI, and 7EDUC+4EDUC': {
        '7CORR': ['7CORR'],
        '7SCIEN': ['7SCIEN'],
        '7EDUC+4EDUC': ['7EDUC', '4EDUC']
    },
    '7CORR + 1AMITY': {
        '7CORR': ['7CORR'],
        '1AMITY': ['1AMITY']
    },
    '7CORR + 1ALLY': {
        '7CORR': ['7CORR'],
        '1ALLY': ['1ALLY']
    },
    '7CORR + 1DIPLOMACY': {
        '7CORR': ['7CORR'],
        'DIPLOMACY': ['1ALLY', '1AMITY', '1ARMCO', '1CHART', '1DISPU', '1ESTAB', '1HEAD', '1OCCUP', '1OPTC', '1PEACE', '1RECOG', '1REPAR', '1STATU', '1TERRI', '1TRUST']
    },
    '7CORR + 2WELFARE': {
        '7CORR': ['7CORR'],
        '2WELFARE': [ '2HEW', '2HUMAN','2LABOR', '2NARK', '2REFUG', '2SANIT', '2SECUR', '2WOMEN' ]
    },
    '7CORR + 3ECONOMIC': {
        '7CORR': ['7CORR'],
        'ECONOMIC': ['3CLAIM', '3COMMO', '3CUSTO', '3ECON', '3INDUS', '3INVES', '3MOSTF', '3PATEN', '3PAYMT', '3PROD', '3TAXAT', '3TECH', '3TOUR','3TRADE','3TRAPA']
    },
    '7CORR + 4AID': {
        '7CORR': ['7CORR'],
        '4AID': ['4AGRIC','4AID', '4ATOM', '4EDUC', '4LOAN', '4MEDIC', '4MILIT', '4PCOR', '4RESOU', '4TECA', '4UNICE']
    },
    '7CORR + 5TRANSPORT': {
        '7CORR': ['7CORR'],
        '5TRANSPORT': ['5AIR', '5LAND', '5TRANS', '5WATER']
    },
    '7CORR + 6COMMUNICATIONS': {
        '7CORR': ['7CORR'],
        '6COMMUNICATIONS': ['6COMMU', '6MEDIA', '6POST', '6TELCO']
    },
    '7CULTURE': {
        '7CULT': ['7CULT'],
        '7CORR': ['7CORR'],
        '7EDUC': ['7EDUC'],
        '7RELIG': ['7RELIG'],
        '7SCIEN': ['7SCIEN'],
        '7SEMIN': ['7SEMIN'],
        '7SPACE': ['7SPACE']
    },
    '7CULT+7CORR': {
        '7CULT+7CORR': ['7CULT', '7CORR'],
        '7CULT': ['7CULT'],
        '7CORR': ['7CORR']        
    },
    '7CORR + 8RESOURCES': {
        '7CORR': ['7CORR'],
        '8RESOURCES': ['8AGRIC', '8CATTL', '8ENERG', '8ENVIR', '8FISH', '8METAL', '8WATER', '8WOOD']
    },
    '7CORR + 9ADMINISTRATION': {
        '7CORR': ['7CORR'],
        '9ADMINISTRATION': ['9ADMIN', '9BOUND', '9CITIZ', '9CONSU', '9LEGAL', '9MILIT', '9MILMI', '9PRIVI', '9VISAS', '9XTRAD' ]
    },
}

category_group_maps = { 
    category_name: { v: k for k in category_group_settings[category_name].keys() for v in category_group_settings[category_name][k]  }
        for category_name in category_group_settings.keys()
}

static_color_map = StaticColorMap()

def plot_display_quantity_of_topics(
    pivot, kind, stacked, xlabel='', ylabel='', plot_style='classic', figsize=(12,10), ylim=None, **kwargs
):
    global static_color_map
    matplotlib.style.use(plot_style)
    colors = static_color_map.get_palette(pivot.columns)

    ax = pivot.plot(kind=kind, stacked=stacked, figsize=figsize, color=colors, **kwargs)

    ax.set_ylabel(ylabel)
    ax.set_xlabel(xlabel)
    
    if ylim:
        ax.set_ylim(ylim)
        
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    
    legend = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    legend.get_frame().set_linewidth(0.0)
    
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
        
def get_quantity_of_categorized_treaties(treaties, period_group, topic_category, recode_is_cultural):
    
    df = treaties[treaties[period_group]!='other']
    
    if recode_is_cultural:
        df.loc[df.is_cultural, 'topic1'] = '7CORR'
        
    df['topic_category'] = df.apply(lambda x: topic_category.get(x['topic1'], 'OTHER'), axis=1)
    
    return df

def display_quantity_of_topics(
    period_group=None,
    topic_category=None,
    party_group=None,
    recode_is_cultural=False,
    normalize_values=False,
    extra_other_category=False,
    output_format='chart',
    plot_style='classic'
    ):
    try:
        
        parties = party_group['parties']
        
        categorized_treaties = get_quantity_of_categorized_treaties(state.treaties, period_group, topic_category, recode_is_cultural)
        
        if not extra_other_category:
            categorized_treaties = categorized_treaties[categorized_treaties.topic_category!='OTHER']
        
        if len(parties) == 0:
            print('Please select one or more parties!')
        
        mask = (categorized_treaties.party1.isin(parties)|(categorized_treaties.party2.isin(parties)))
        if party_group['label'] == 'ALL':
            parties_treaties = categorized_treaties
        elif party_group['label'] == 'ALL OTHER':
            parties_treaties = categorized_treaties.loc[~mask]
        else:
            parties_treaties = categorized_treaties.loc[mask]
        
        if parties_treaties.shape[0] == 0:
            print('No data for: ' + ','.join(parties))
            return
        
        data = parties_treaties\
                .groupby([period_group, 'topic_category'])\
                .size()\
                .reset_index()\
                .rename(columns={ period_group: 'Period', 'topic_category': 'Category', 0: 'Count' })

        pivot = pd.pivot_table(data, index=['Period'], values=["Count"], columns=['Category'], fill_value=0)
        pivot.columns = [ x[-1] for x in pivot.columns ]
        
        if period_group == 'signed_year':
            missing_years = [ x for x in range(data.Period.min(), data.Period.max() + 1) if x not in pivot.index ]
            for year in missing_years:
                pivot.loc[year] = len(pivot.columns) * [0]
            pivot.sort_index(axis=0, inplace=True)

        if normalize_values is True:
            pivot = pivot.div(0.01 * pivot.sum(1), axis=0)

        if output_format.startswith('plot'):

            label1 = 'Number of treaties' if not normalize_values else 'Share%'
            title = party_group['label']
            
            ylabel = label1 if 'barh' not in output_format else '' 
            xlabel = label1 if 'barh' in output_format else ''

            stacked = 'stacked' in output_format
            kind = output_format.split('_')[1]
            height = 10 if 'barh' in output_format and period_group == 'signed_year' else 6
            ylim = [0, 100] if normalize_values else None

            plot_display_quantity_of_topics(
                pivot, kind=kind, stacked=stacked, xlabel=xlabel, ylabel=ylabel, plot_style=plot_style, ylim=ylim, figsize=(16,height), title=title
            )

        elif output_format == 'table':
            print(party_group.label)
            display(data)
        else:
            print(party_group.label)
            display(pivot)
    except Exception as ex:
        logger.error(ex)
        # raise
        
def treaty_quantities_by_selected_topics_main():

    wc = BaseWidgetUtility(
        period_group=widgets.Dropdown(
            options= { x['title']: x['column'] for x in state.period_specification },
            value='signed_period',
            description='Divisions',
            layout=widgets.Layout(width='200px')
        ),
        topic_category_name=widgets.Dropdown(
            options=category_group_maps.keys(),
            value='7CULTURE',
            description='Category:',
            layout=widgets.Layout(width='200px')
        ),
        recode_is_cultural=widgets.ToggleButton(
            description='Recode 7CORR',
            tooltip='Recode all treaties with cultural=yes as 7CORR',
            value=True,
            layout=widgets.Layout(width='120px')
        ),
        normalize_values=widgets.ToggleButton(
            description='Normalize%',
            tooltip='Display shares per category instead of count',
            layout=widgets.Layout(width='120px')
        ),
        output_format=widgets.Dropdown(
            description='Output',
            value='plot_bar_stacked',
            options=config.output_formats,
            layout=widgets.Layout(width='200px')
        ),
        plot_style=widgets.Dropdown(
            options=config.matplotlib_plot_styles,
            value='seaborn-pastel',
            description='Style:',
            layout=widgets.Layout(width='200px')
        ),   
        parties=widgets.SelectMultiple(
            options=state.get_countries_list(),
            value=['FRANCE'],
            rows=10,
            description='Parties',
            disabled=False,
            layout=widgets.Layout(width='180px')
        ),
        chart_per_party=widgets.ToggleButton(
            description='Chart per party',
            tooltip='Display one chart per party',
            layout=widgets.Layout(width='120px')
        ),
        extra_other_category=widgets.ToggleButton(
            description='Add OTHER topics',
            tooltip='Add summed up category "OTHER" for all other topic (and for selected parties)',
            layout=widgets.Layout(width='120px'),
            value=False
        )
    )
    
    def display_quantity_of_topics_proxy(
        period_group,
        topic_category_name,
        recode_is_cultural=False,
        normalize_values=False,
        extra_other_category=None,
        output_format='chart',
        plot_style='classic',
        parties=None,
        chart_per_party=False
    ):
        if not chart_per_party:
            parties = [ x for x in parties if x not in ['ALL', 'ALL OTHER' ] ]
            party_groups = [
                {
                    'label': ', '.join(parties[:6]) + ('' if len(parties) <= 6 else ' +{} parties'.format(len(parties)-6)),
                    'parties': parties
                }
            ]
        else:
            party_groups = [
                {
                    'label': x,
                    'parties': [ x ] if x != 'ALL OTHER' else parties
                } for x in parties
            ] 
        
        for party_group in party_groups:
            topic_category = category_group_maps[topic_category_name]
            display_quantity_of_topics(
                period_group=period_group,
                topic_category=topic_category,
                recode_is_cultural=recode_is_cultural,
                normalize_values=normalize_values,
                extra_other_category=extra_other_category,
                output_format=output_format,
                plot_style=plot_style,
                party_group=party_group
            )
        
    itw = widgets.interactive(
        display_quantity_of_topics_proxy,
        period_group=wc.period_group,
        topic_category_name=wc.topic_category_name,
        parties=wc.parties,
        recode_is_cultural=wc.recode_is_cultural,
        normalize_values=wc.normalize_values,
        extra_other_category=wc.extra_other_category,
        output_format=wc.output_format,
        plot_style=wc.plot_style,
        chart_per_party=wc.chart_per_party
    )

    boxes = widgets.HBox(
        [
            widgets.VBox([ wc.period_group, wc.topic_category_name]),        
            widgets.VBox([ wc.parties]),
            widgets.VBox([ wc.recode_is_cultural, wc.extra_other_category]),
            widgets.VBox([ wc.output_format, wc.plot_style ]),
            widgets.VBox([ wc.normalize_values, wc.chart_per_party ])
        ]
    )
    display(widgets.VBox([boxes, itw.children[-1]]))
    itw.update()
    
treaty_quantities_by_selected_topics_main()